## Demographic Analysis

## Datasets

## Dataset and documentation: 
https://api.census.gov/data/2020/dec/pl.html
https://www.youtube.com/watch?v=vv4MZgqgHe8

In [109]:
RACE_POP_COUNT = 'RacePopCount.json'
ETHNIC_COUNT = 'EthnicPopCount.json'
datasets_and_uris = {}
datasets_and_uris[RACE_POP_COUNT] = "http://api.census.gov/data/2020/dec/pl"
datasets_and_uris[ETHNIC_COUNT] = "http://api.census.gov/data/2020/dec/pl"

#ASSETS_PATH = 'RacePopCount'

### Required Packages

In [57]:
from collections import Counter
import numpy as np
import pandas as pd
import altair as alt
from altair import datum
import json
import requests
import random
import time
import re
import requests
import os
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)


## Downloading Required Data

In [120]:
def createPredicatesRaceandEthnicityPopCount(tableId, var_cnt):
  '''
  create predicates for the census data API
  base_uri - This is the base uri for the census API
  tableId - table name, which we want to download
  var_cnt - Number of variables the table has 
  '''
  county = "*" #"099,163,125" - These are for Macomb, Wayne, Oakland specifically
  state= "26"
  estimate = []
  #moe = []
  for i in range(1, var_cnt+1):
    estimate.append(tableId + '_00' + str(i) + 'N')
    #moe.append(tableId + '_00' + str(i) + 'M')

  get_vars = estimate + ['NAME'] #+ moe
  predicates = {}
  predicates["get"] = ','.join(get_vars)
  predicates["for"] = "county:" + county
  predicates["in"] = "state:" + state
  
  return(predicates)

#def createPredicatesAge(county, token):
  #  '''
  #create predicates/headers for the CHAS data API
  #  county - County id for which we want to download the cost burden
  #  token - API token to access the CHAS API, CHN needs to register for CHAS API and get this token 
   # '''
    #predicates = {}
    #predicates['type'] = 3
    #predicates['stateId'] = 26
    ##predicates['entityId'] = county
    #auth_token_string = "Bearer "+ token
    #headers = {"Authorization": auth_token_string}
    #return(predicates, headers)


def download_data(base_uri,predicates, path, filename, headers=''):
    filepath = path + filename
    if os.path.isfile(filepath):
        print(filepath)
        print('file already exists, clearing and updating')
        open(filepath, 'w').close()
    #else:
        result = requests.get(base_uri, params=predicates, headers=headers) 
        if result.status_code == 200:
          with open(filepath, 'wb') as f:
              f.write(result.content)
        else:
          print('API returned Erro: ', result.status_code, '\n', 'Downloading of data failed.',  \
                'Please check the API')
    else: 
        result = requests.get(base_uri, params=predicates, headers=headers) 
        if result.status_code == 200:
          with open(filepath, 'wb') as f:
              f.write(result.content)
        else:
          print('API returned Erro: ', result.status_code, '\n', 'Downloading of data failed.',  \
                'Please check the API')

In [121]:
#Download Race Population Counts
tableId = 'P1'
var_cnt = 9
predicates = createPredicatesRaceandEthnicityPopCount(tableId, var_cnt)
ASSETS_PATH = ''
download_data(datasets_and_uris[RACE_POP_COUNT],predicates, ASSETS_PATH, RACE_POP_COUNT)

RacePopCount.json
file already exists, clearing and updating


In [122]:
#Download Ethnicity Counts Data
#Total Population Count; Hispanic or Latino of any race (Population Count); Non-Hispanic/Non-Latino (Population Count)
tableId = 'P2'
var_cnt = 3
predicates = createPredicatesRaceandEthnicityPopCount(tableId, var_cnt)
ASSETS_PATH = ''
download_data(datasets_and_uris[ETHNIC_COUNT],predicates, ASSETS_PATH, ETHNIC_COUNT)


EthnicPopCount.json
file already exists, clearing and updating


## Data Transformations - One Race Population Count and Two or More Races Total Population Count

In [123]:
race_pop_count_df = pd.read_json('RacePopCount.json')
race_pop_count_df.rename(columns={0:'total', 1:'one_tot',2:'one_wht', 3:'one_blk',
4:'one_ind', 5:'one_asn', 6:'one_pacific', 7:'one_other', 8:'two__more_total', 9:'NAME', 10:'state', 11:'county'}, inplace=True)
race_pop_count_df = race_pop_count_df.iloc[1:, :]
race_pop_count_df['NAME'] = race_pop_count_df['NAME'].str.split(',', expand=True).iloc[:, 0]
race_pop_count_df.drop(columns=['state', 'county'], inplace=True)

In [124]:
race_pop_count_df.head()

,total,one_tot,one_wht,one_blk,one_ind,one_asn,one_pacific,one_other,two__more_total,NAME
1,10167,9785,9644,23,61,14,1,42,382,Alcona County
2,8842,8351,7363,587,328,26,1,46,491,Alger County
3,120502,112970,105564,1584,799,874,22,4127,7532,Allegan County
4,28907,27656,27177,99,122,119,12,127,1251,Alpena County
5,23431,22372,21902,39,201,72,20,138,1059,Antrim County


## Data Transformations - Ethnic Population Count Hispanic and Non-Hispanic

In [125]:
ethnic_pop_count_df = pd.read_json('EthnicPopCount.json')

In [126]:
ethnic_pop_count_df.head()

,0,1,2,3,4,5
0,P2_001N,P2_002N,P2_003N,NAME,state,county
1,10167,122,10045,"Alcona County, Michigan",26,001
2,8842,115,8727,"Alger County, Michigan",26,003
3,120502,9389,111113,"Allegan County, Michigan",26,005
4,28907,417,28490,"Alpena County, Michigan",26,007
